In [38]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import box
from scipy.spatial.distance import cdist

In [39]:
# read conflict data
df = pd.read_csv("data/ACLED Data_2025-11-22.csv")
study_regions = ["Tigray", "Amhara"]
conflict_df = df[df["admin1"].isin(study_regions)]

In [40]:
df["sub_event_type"].value_counts()

sub_event_type
Armed clash                            4163
Attack                                 1804
Abduction/forced disappearance          207
Government regains territory            178
Non-state actor overtakes territory     149
Sexual violence                         121
Violent demonstration                    85
Mob violence                             63
Name: count, dtype: int64

In [41]:
# read kebele data
gdf = gpd.read_file("data/Ethiopia_AdminBoundaries.shp").to_crs(32637)
kebele_gdf = gdf[gdf["R_NAME"].isin(study_regions)]

In [42]:
# Build uniform 1° grid for study area bounds
min_x, min_y, max_x, max_y = [round(coord) for coord in kebele_gdf.total_bounds.tolist()]
cell_size = 25000
x_cells = int((max_x - min_x) / cell_size)
y_cells = int((max_y - min_y) / cell_size)

polygons = []
tile_ids = []
for i in range(x_cells):
    for j in range(y_cells):

        x1 = min_x + i * cell_size
        y1 = min_y + j * cell_size

        poly = box(x1, y1, x1 + cell_size, y1 + cell_size)
        tile_id = f"{x1}_{y1 + cell_size}"
        
        polygons.append(poly)
        tile_ids.append(tile_id)

tiles = gpd.GeoDataFrame({"tile_id": tile_ids, "geometry": polygons}, crs="EPSG:32637")
tiles['centroid_lon'] = tiles.geometry.centroid.x
tiles['centroid_lat'] = tiles.geometry.centroid.y

In [43]:
len(tiles)

567

In [44]:
conflict_gdf = gpd.GeoDataFrame(
    conflict_df,
    geometry=gpd.points_from_xy(conflict_df.longitude, conflict_df.latitude),
    crs='EPSG:4326'
)
conflict_gdf = conflict_gdf.to_crs(32637)
conflict_gdf = conflict_gdf.sjoin(tiles, how='left', predicate='within')

In [45]:
# Create MultiIndex with all tile-year combinations
all_tiles = tiles['tile_id'].unique()
all_years = conflict_gdf['year'].unique()
full_index = pd.MultiIndex.from_product([all_tiles, all_years], names=['tile_id', 'year'])

# Aggregate conflicts
casualty_summary = conflict_gdf.groupby(['tile_id', 'year'])['fatalities'].sum()

# Reindex to include all combinations
casualty_summary = casualty_summary.reindex(full_index, fill_value=0).reset_index()

# Merge with tiles geometry
casualty_tiles_gdf = tiles.merge(casualty_summary, on='tile_id', how='right')
casualty_tiles_gdf = gpd.GeoDataFrame(casualty_tiles_gdf, geometry='geometry', crs=tiles.crs)

In [46]:
casualty_tiles_gdf

,tile_id,geometry,centroid_lon,centroid_lat,year,fatalities
0,93946_988825,"POLYGON ((118946 963825, 118946 988825, 93946 ...",106446.0,976325.0,2020,0
1,93946_988825,"POLYGON ((118946 963825, 118946 988825, 93946 ...",106446.0,976325.0,2021,0
2,93946_988825,"POLYGON ((118946 963825, 118946 988825, 93946 ...",106446.0,976325.0,2022,0
3,93946_988825,"POLYGON ((118946 963825, 118946 988825, 93946 ...",106446.0,976325.0,2023,0
4,93946_988825,"POLYGON ((118946 963825, 118946 988825, 93946 ...",106446.0,976325.0,2024,0
...,...,...,...,...,...,...
2830,593946_1638825,"POLYGON ((618946 1613825, 618946 1638825, 5939...",606446.0,1626325.0,2020,0
2831,593946_1638825,"POLYGON ((618946 1613825, 618946 1638825, 5939...",606446.0,1626325.0,2021,0
2832,593946_1638825,"POLYGON ((618946 1613825, 618946 1638825, 5939...",606446.0,1626325.0,2022,0
2833,593946_1638825,"POLYGON ((618946 1613825, 618946 1638825, 5939...",606446.0,1626325.0,2023,0


In [47]:
# Parameters
alpha = 1  # distance decay parameter
D_max = 50  # maximum distance in km

def calculate_conflict_intensity_vectorized(tiles_gdf, conflict_gdf, year, alpha=1, D_max=50):
    """Vectorized version - much faster"""
    
    year_conflicts = conflict_gdf[conflict_gdf['year'] == year].copy()
    
    if len(year_conflicts) == 0:
        return np.zeros(len(tiles_gdf))
    
    # Reproject to metric CRS
    tiles_metric = tiles_gdf.to_crs('EPSG:3857')
    conflicts_metric = year_conflicts.to_crs('EPSG:3857')
    
    # Get coordinates
    tile_coords = np.array([[geom.centroid.x, geom.centroid.y] 
                            for geom in tiles_metric.geometry])
    conflict_coords = np.array([[geom.x, geom.y] 
                                for geom in conflicts_metric.geometry])
    
    # Distance matrix in km
    distances_km = cdist(tile_coords, conflict_coords) / 1000
    
    # Get fatalities
    fatalities = year_conflicts['fatalities'].values
    
    # Apply distance threshold
    mask = (distances_km <= D_max) & (distances_km > 0)
    
    # Calculate intensity (avoid division by zero)
    distances_km[distances_km == 0] = 0.1  # Small distance for events at centroid
    
    # Vectorized calculation: (deaths_j / distance^alpha) where distance <= D_max
    weighted_fatalities = np.where(mask, fatalities / (distances_km ** alpha), 0)
    
    # Sum across all conflicts for each tile
    intensity = weighted_fatalities.sum(axis=1)
    
    return intensity

In [48]:
years = sorted(conflict_gdf['year'].unique())
results = []

for year in years:
    intensities = calculate_conflict_intensity_vectorized(tiles, conflict_gdf, year, alpha=alpha, D_max=D_max)
    
    year_result = tiles[['tile_id', 'geometry']].copy()
    year_result['year'] = year
    year_result['conflict_intensity'] = intensities
    
    results.append(year_result)

conflict_intensity_gdf = gpd.GeoDataFrame(
    pd.concat(results, ignore_index=True),
    crs=tiles.crs
)

In [49]:
conflict_intensity_gdf_4326 = conflict_intensity_gdf.to_crs(4326)
conflict_intensity_gdf_4326[conflict_intensity_gdf_4326["tile_id"] == "493946_1473825"].geometry

GeoSeries([], Name: geometry, dtype: geometry)

In [50]:
conflict_intensity_df = conflict_intensity_gdf.drop("geometry", axis=1)
conflict_intensity_df

,tile_id,year,conflict_intensity
0,93946_988825,2020,0.0
1,93946_1013825,2020,0.0
2,93946_1038825,2020,0.0
3,93946_1063825,2020,0.0
4,93946_1088825,2020,0.0
...,...,...,...
2830,593946_1538825,2024,0.0
2831,593946_1563825,2024,0.0
2832,593946_1588825,2024,0.0
2833,593946_1613825,2024,0.0


In [51]:
conflict_intensity_df.to_csv("conflict_intensity.csv")